# Using inaFaceAnalyzer API: advanced tutorial
In this tutorial, define a custom analysis pipeline. This is achieved by defining 3 core parametric elements:
* face detection
* face classification
* image or video processing engine

Both of these processing classes are are designed as ['functions objects' or 'functors'](https://en.wikipedia.org/wiki/Function_object): instances of these objects can be used as functions, executing the code implemented in `__call__` methods.


## Install `inaFaceAnalyzer`

In [ ]:
# try to import inaFaceAnalyzer and import it from Pypi's
# if it is not available
try:
  import inaFaceAnalyzer
except:
  # install inaFaceAnalyzer Pypi's distribution
  !pip install inaFaceAnalyzer

## Download and display sample video
still easy...

In [ ]:
# allow to display videos in jupyter and collab notebooks
from inaFaceAnalyzer.display_utils import notebook_display_local_vid
# used to download remote files
from tensorflow.keras.utils import get_file


# donwload remote file
sample_vid = get_file('samplevid.mp4', 'https://github.com/ina-foss/inaFaceAnalyzer/raw/master/media/pexels-artem-podrez-5725953.mp4')
#display local file path
print(sample_vid)
# display video
notebook_display_local_vid(sample_vid)

## Face classification model
`inaFaceAnalyzer` implements four face classification models.
Both of them require eye-aligned images of isolated faces with fixed dimension (224\*224). This face dimension size may be changed when integrating future face classifiers.

Resnet50FairFaceGRA is the most accurate model, and is able to detect gender and age from face. It is used by default in processing engines. The remaining models do only detect gender.

In [ ]:
# import all available face classifier classes listed bellow
from inaFaceAnalyzer.face_classifier import Resnet50FairFaceGRA, Resnet50FairFace, Vggface_LSVM_FairFace, Vggface_LSVM_YTF

In [ ]:
# download and decode remote 224*224 preprocessed faces

from inaFaceAnalyzer.opencv_utils import imread_rgb

diallo_fname = get_file('diallo224.jpg', 'https://github.com/ina-foss/inaFaceAnalyzer/raw/master/media/diallo224.jpg')
knuth_fname = get_file('knuth224.jpg', 'https://github.com/ina-foss/inaFaceAnalyzer/raw/master/media/knuth224.jpg')

# open images and convert them to numpy RBG arrays
# verbose=True allow to display the images being opened
diallo_img = imread_rgb(diallo_fname, verbose=True)
knuth_img = imread_rgb(knuth_fname, verbose=True)
# print the resulting opened image as numpy array
print(diallo_img.shape)
print(diallo_img)

In [ ]:
# create a classifier instance: Resnet50GairFaceGRA is the most accurate model
c = Resnet50FairFaceGRA()

In [ ]:
# preprocess_img_list method allow to process a list of preprocessed image files FAST
# it is used when evaluating a classifer on preprocessed faces
c.preprocessed_img_list([diallo_fname, knuth_fname])

In [ ]:
# process the same list of images with another classifier that do only predict gender
c = Vggface_LSVM_YTF()
c.preprocessed_img_list([diallo_fname, knuth_fname])

In [ ]:
# When used as a functor, a classifier requires a list of opened images as first argument
# named argument verbose is not to be used in production and allow to display the images being classified
c([diallo_img, knuth_img], verbose=True)

## Defining the Face Detection module
The face detection module is in charge of finding faces in an image frame. Face detection classes return one or several bounding boxes associated to a face detection confidence.


Two face detection classes are provided :
* `LibFaceDetection` (default) : is the most recent face detection engine integrated. It can take advantage of GPU acceleration and is able de detect the smallest faces. It may be slow when used with high resolution images.
* `OcvCnnFaceDetector` : is based on OpenCV CNN face detection model. Images are fist resized to 300\*300 pixels, which may result in missing the smallest faces. It is definitely faster.


Implemented face detection classes allow to define 4 parameters in their constructor
 * `minconf` : the minimal face detection confidence for being returned (default values dependent on the face detection class choosen).
 * `min_size_px` : minimal face size in pixels (default 30): better classification results requires face sizes above 75 pixels
 * `min_size_prct` : minimal face size as a percentage of frame minimal dimension
 * `padd_prc` : percentage of black padding pixels to be applied on images before detection (fault values are set or each detection class).

In [ ]:
# import all available face detection classes listed bellow
from inaFaceAnalyzer.face_detector import LibFaceDetection, OcvCnnFacedetector

In [ ]:
# download and open a sample image
india_fname = get_file('india.jpg', 'https://github.com/ina-foss/inaFaceAnalyzer/raw/master/media/800px-India_(236650352).jpg')
img = imread_rgb(india_fname, verbose=True)

In [ ]:
# create a detector instance with default parameters
d = OcvCnnFacedetector()
# detect faces and return Detection named tuples with fields bbox and detect_conf
d(img)

In [ ]:
# use verbose=True to display detected faces
d(img, verbose=True)

In [ ]:
# create a detector instance that will discard faces below 100 pixels
d = OcvCnnFacedetector(min_size_px=100)
d(img, verbose=True)

## Defining an analysis engine
Four analysis engine are provided : 
* `ImageAnalyzer` : which is used with image files
* `VideoAnalyzer` : default choice to be used with video
* `VideoKeyframes` : do only process video Keyframes
* `VideoTracking` : perform face detection and tracking, allowing to faster computations and smooth results

Engine constructors accept at least 3 optional arguments : 
* `face_detector` : instance of face detection model
* `face_classifier` : instance of face classification model
* `verbose` : when set to True, display intermediate images and results (not to be used in production)

In [ ]:
# We import here 3 different Video processing engine, and 1 image processing engine
# both inherits from inaFaceAnalyzer.FaceAnalyzer
from inaFaceAnalyzer.inaFaceAnalyzer import VideoAnalyzer, VideoKeyframes, VideoTracking, ImageAnalyzer

In [ ]:
# we instantiate an Image Analyzer with custom face detection and face classification module
classifier = Vggface_LSVM_FairFace()
detector = OcvCnnFacedetector(minconf=0.5)
ia = ImageAnalyzer(face_classifier=classifier, face_detector=detector, verbose=True)
ia(india_fname)

In [ ]:
# We create an image analyzer instance with different face classification and detection models
detector = LibFaceDetection(minconf=0.4)
classifier = Resnet50FairFaceGRA()
ia = ImageAnalyzer(face_classifier=classifier, face_detector=detector, verbose=True)
ia(india_fname)

In [ ]:
# using video engine with verbosity
va = VideoAnalyzer(face_classifier=Resnet50FairFaceGRA(), face_detector=OcvCnnFacedetector(), verbose=True)
# will set FPS at 0.5, meaning 0.5 frame analyzed per second of content. ie: 1 frame every 2 seconds
va(sample_vid, fps=0.5)

In [ ]:
# here we analyze 5 frames per second
va(sample_vid, fps=5)

In [ ]:
# Using the video keyframes engine
# analysis based on keyframes alllow to get a fast summary of a video,
# associated to a variable frame analysis rate
# it cannot be used with fps argument
va = VideoKeyframes(face_detector=LibFaceDetection(), face_classifier=Resnet50FairFaceGRA(), verbose=True)
# the video keyframes engine is run with verbose = True allowing to show each processed frames
# For this material, key frames coorespond to frames 0, 91, 182 and 273
va(sample_vid)

In [ ]:
# Face tracking can be used to lower computation time and smooth prediction results
# VideoTracking constructor require a detection period_argument, defining how often
# the detection engine will be used
# in this example, we analyze all frames (30 per seconds), with a detection_period = 5
# Face detection procedure will be used 6 times per second
#
# Resulting dataframes have additional columns
# * face_id: numeric identifier allowing to track previously detected faces
# * columns with avg suffix: smoothed result
ta = VideoTracking(detection_period=5, face_classifier=Resnet50FairFaceGRA())
ta(sample_vid)